In [1]:
from __future__ import print_function
import os, shutil
import argparse

import cv2
import rosbag
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from tqdm import tqdm
from glob import glob
import roslz4

In [2]:
bag_dir = '/data/handhygiene/bagfiles/'
img_dir = '/data/handhygiene/images/'
bags = sorted(glob(bag_dir + '*.bag'), key=os.path.getmtime)
add_data_from = 0

In [ ]:
for i, bag in enumerate(tqdm(bags)):
    date = bag.split('/')[-1].split('_')[0]
    rosbag = rosbag.Bag(bag, "r", allow_unindexed=True)
    bridge = CvBridge()
    count = 0
    rgb_img_topic = '/device_0/sensor_1/Color_0/image/data'
            
    for topic, msg, t in rosbag.read_messages(topics=[rgb_img_topic]):
        if msg is not None:
            cv_img = bridge.imgmsg_to_cv2(msg, "bgr8")

            if not os.path.exists(img_dir):
                os.path.mkdir(img_dir)
            if add_data_from == 0:
                data_num = i+1
            else :
                data_num = add_data_from
            target_dir = os.path.join(img_dir, "{}_{}_frames{:06d}.jpg".format(data_num, date, count))
            cv2.imwrite(target_dir, cv_img)
            print("Wrote image {} to {}".format(count, target_dir))
        shutil.move(bag, os.path.join(bag, 'extracted'))
        count += 1
    rosbag.close()

In [ ]:
#for dir in bag_sub_dirs:
#    i = 1
#    dir_rgb = dir + 'rgb_images{}'.format(i)
#    !cd $dir_rgb && pwd && mkdir $dir/videos
#    !ffmpeg -r 15 -i $dir_rgb/frame%06d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $dir/videos/output.mp4 -y